In [ ]:
import requests

remote_url = "https://raw.githubusercontent.com/hhhuang/nlp2019fall/master/classification/"

r = requests.get(remote_url + "train.csv", allow_redirects=True)
open('stance-detection-train.csv', 'wb').write(r.content)

r = requests.get(remote_url + "test.csv", allow_redirects=True)
open('stance-detection-test.csv', 'wb').write(r.content)


500660

In [ ]:
import csv
from collections import Counter

# Training data

fin = open("/content/stance-detection-train.csv")
# Test data
fin = open("/content/stance-detection-test.csv")


reader = csv.DictReader(fin)
labels = Counter()
targets = Counter()
for line in reader:
    labels[line['Stance']] += 1
    targets[line['Target']] += 1
print(labels)
print(targets)

Counter({'AGAINST': 1014, 'NONE': 490, 'FAVOR': 452})
Counter({'Donald Trump': 707, 'Hillary Clinton': 295, 'Feminist Movement': 285, 'Legalization of Abortion': 280, 'Atheism': 220, 'Climate Change is a Real Concern': 169})


In [ ]:
!pip install transformers==4.28.0
!pip install datasets
!pip install torch
!pip install tqdm
!pip install evaluate
!pip install pandas
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import csv
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from collections import defaultdict, Counter

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoConfig
from transformers import TrainingArguments, Trainer
from transformers import get_scheduler

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch import nn

from datasets import Dataset
import evaluate
import pandas as pd
import numpy as np

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Assignment III 修改區
請修改下方的程式碼，使得 F1 越高越好

In [ ]:
from transformers import TextClassificationPipeline

class StanceDetectionModel:
    def __init__(self):
        self.test_data = "/content/stance-detection-test.csv"
        self.training_data = "/content/stance-detection-train.csv"
        self.model_path = "trained_model"
        self.stance_label_mapping = {'AGAINST': 0, 'NONE': 1, 'FAVOR': 2}
        self.inverted_stance_label_mapping = {v: k for k, v in self.stance_label_mapping.items()}

    def create_model(self, load_trained=False):
        pretrained_model = "roberta-base"
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
        self.model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, num_labels=3)

    def load_tweet_data(self, filename):
        """ Data source: https://alt.qcri.org/semeval2016/task6/index.php?id=data-and-tools """
        import csv
        reader = csv.DictReader(open(filename))    # The original released dataset was encoded in ISO-8859-1. The attached version has been changed to utf8 for universality.
        cnt = 0
        for row in reader:
            cnt += 1
            labels = [0.0] * 3
            labels[self.stance_label_mapping[row['Stance']]] = 1.0
            yield {'labels': labels, 'text': row['Tweet']}

    def tokenize_dataset(self, data):
        return self.tokenizer(data['text'], return_tensors='pt', truncation=True, padding='max_length', max_length=280)

    def compute_metrics(self, eval_pred):
        logits, labels = eval_pred
        predictions, references = [], []
        for i in range(len(logits)):
            predictions.append(np.argmax(logits[i][:3]))
            references.append(np.argmax(labels[i][:3]))
        #predictions = np.argmax(logits, axis=-1)
        # Following the evaluation of SemEval-2016, only the F-scores of Favour (2) and Against (0) are measured.
        return self.metric.compute(predictions=predictions, references=references, labels=[0, 2], average='macro')

    def train(self):
        #stances = ['Hillary Clinton', 'Feminist Movement', 'Legalization of Abortion', 'Atheism', 'Climate Change is a Real Concern', 'Donald Trump']
        self.create_model()
        load_training_data = lambda: self.load_tweet_data(self.training_data)
        load_test_data = lambda: self.load_tweet_data(self.test_data)
        training_data = Dataset.from_generator(load_training_data).shuffle(seed=42).map(self.tokenize_dataset, batched=True)
        test_data = Dataset.from_generator(load_test_data).map(self.tokenize_dataset, batched=True)

        training_args = TrainingArguments(
            output_dir="test_trainer",
            evaluation_strategy="epoch",
            num_train_epochs=10,
            per_device_train_batch_size=24,
            per_device_eval_batch_size=24,
        )

        self.metric = evaluate.load("f1")
        self.trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=training_data,
            eval_dataset=test_data,
            compute_metrics=self.compute_metrics,
        )
        self.trainer.train()

    def evaluate(self):
        return self.trainer.evaluate()

    def predict(self, instances):
        pipe = TextClassificationPipeline(model=self.model, tokenizer=self.tokenizer, return_all_scores=True)
        predictions = []
        for results in pipe([ {"text": instance["text"]} for instance in instances]):
            predictions.append(max(
                [(result['score'], self.inverted_stance_label_mapping[int(result['label'][-1])]) for result in results if result['label'] in ['LABEL_0', 'LABEL_1', 'LABEL_2']])[1])
        return predictions

In [ ]:
model = StanceDetectionModel()
model.train()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /root/.cache/huggingface/datasets/generator/default-0c6ddc9fbf507ada/0.0.0. Subsequent calls will reuse this data.


Map:   0%|          | 0/2914 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset generator downloaded and prepared to /root/.cache/huggingface/datasets/generator/default-46779dfa02a92ec6/0.0.0. Subsequent calls will reuse this data.


Map:   0%|          | 0/1956 [00:00<?, ? examples/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│ in train:64                                                                                      │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1662 in train                    │
│                                                                                                  │
│   1659 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1660 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1661 │   │   )                                                                                 │
│ ❱ 1662 │   │   return inner_training_loop(                                                       │
│   1663 │   │   │   args=args,                                                                    │
│   1664 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1665 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1929 in _inner_training_loop     │
│                                                                                                  │
│   1926 │   │   │   │   │   with model.no_sync():                                                 │
│   1927 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1928 │   │   │   │   else:                                                                     │
│ ❱ 1929 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1930 │   │   │   │                                                                             │
│   1931 │   │   │   │   if (                                                                      │
│   1932 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2699 in training_step            │
│                                                                                                  │
│   2696 │   │   │   return loss_mb.reduce_mean().detach().to(self.args.device)                    │
│   2697 │   │                                                                                     │
│   2698 │   │   with self.compute_loss_context_manager():                                         │
│ ❱ 2699 │   │   │   loss = self.compute_loss(model, inputs)                                       │
│   2700 │   │                                                                                     │
│   2701 │   │   if self.args.n_gpu > 1:                                                           │
│   2702 │   │   │   loss = loss.mean()  # mean() to average on multi-gpu parallel training        │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2731 in compute_loss             │
│                                                                                                  │
│   2728 │   │   │   labels = inputs.pop("labels")                                                 │
│   2729 │   │   else:                                                                             │
│   2730 │   │   │   labels = None                                                                 │
│ ❱ 2731 │   │   outputs = model(**inputs)                   

# Assignment III: 目標1

請修改 `Assignment III 修改區` 那格，使得下方回報之 F1 越高越好

In [ ]:
print(model.evaluate()['eval_f1'])

0.5900919093568529


# Assignment III: 目標2
請修改 `Assignment III 修改區` 那格，使得下方程式碼能順利執行

In [ ]:
model.predict([
    {"text": "I don't trust Hillary"},
    {"text": "I don't like Trump"},
    {"text": "I like Hillary Clinton. We are personal friends. My support for her candidacy is based on the fact that she is brilliantly qualified for the job."}
    ])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│ in predict:72                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:155 in     │
│ __call__                                                                                         │
│                                                                                                  │
│   152 │   │   │                                                                                  │
│   153 │   │   │   If `top_k` is used, one such dictionary is returned per label.                 │
│   154 │   │   """                                                                                │
│ ❱ 155 │   │   result = super().__call__(*args, **kwargs)                                         │
│   156 │   │   # TODO try and retrieve it in a nicer way from _sanitize_parameters.               │
│   157 │   │   _legacy = "top_k" not in kwargs                                                    │
│   158 │   │   if isinstance(args[0], str) and _legacy:                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1090 in __call__          │
│                                                                                                  │
│   1087 │   │   │   │   final_iterator = self.get_iterator(                                       │
│   1088 │   │   │   │   │   inputs, num_workers, batch_size, preprocess_params, forward_params,   │
│   1089 │   │   │   │   )                                                                         │
│ ❱ 1090 │   │   │   │   outputs = list(final_iterator)                                            │
│   1091 │   │   │   │   return outputs                                                            │
│   1092 │   │   │   else:                                                                         │
│   1093 │   │   │   │   return self.run_multi(inputs, preprocess_params, forward_params, postpro  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/pt_utils.py:124 in __next__       │
│                                                                                                  │
│   121 │   │   │   return self.loader_batch_item()                                                │
│   122 │   │                                                                                      │
│   123 │   │   # We're out of items within a batch                                                │
│ ❱ 124 │   │   item = next(self.iterator)                                                         │
│   125 │   │   processed = self.infer(item, **self.params)                                        │
│   126 │   │   # We now have a batch of "inferred things".                                        │
│   127 │   │   if self.loader_batch_size is not None:                                             │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/pipelines/pt_utils.py:125 in __next__       │
│                                                                                                  │
│   122 │   │                                                                                      │
│   123 │   │   # We're out of items within a batch                                                │
│   124 │   │   item = next(self.iterator)                   